In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
Independent=dataset[['User ID','Gender_Male','Age','EstimatedSalary']]
Independent

,User ID,Gender_Male,Age,EstimatedSalary
0,15624510,1,19,19000
1,15810944,1,35,20000
2,15668575,0,26,43000
3,15603246,0,27,57000
4,15804002,1,19,76000
...,...,...,...,...
395,15691863,0,46,41000
396,15706071,1,51,23000
397,15654296,0,50,20000
398,15755018,1,36,33000


In [7]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'C':[1,10,50,100,500],'kernel':['linear','poly','rbf'],'gamma':['scale','auto']}
grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [1, 10, 50, 100, 500], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf']},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(x_test) #(.predict->defined function)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[73  6]
 [ 5 36]]


In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        79
           1       0.86      0.88      0.87        41

    accuracy                           0.91       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.91      0.91      0.91       120



In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}: 0.9085936101092268


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023118,0.002447,0.013108,0.002337,1,scale,linear,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",0.833024,0.779449,0.644599,0.927778,0.872761,0.811522,0.096561,13
1,0.026552,0.005284,0.011514,0.004371,1,scale,poly,"{'C': 1, 'gamma': 'scale', 'kernel': 'poly'}",0.846429,0.779449,0.646561,0.830519,0.907658,0.802123,0.087887,27
2,0.025043,0.004857,0.017298,0.004938,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.874254,0.874254,0.840114,0.928571,0.946153,0.892669,0.038961,1
3,0.016246,0.003991,0.012471,0.001161,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",0.833024,0.779449,0.644599,0.927778,0.872761,0.811522,0.096561,13
4,0.017234,0.003007,0.008472,0.003556,1,auto,poly,"{'C': 1, 'gamma': 'auto', 'kernel': 'poly'}",0.846429,0.779449,0.646561,0.830519,0.907658,0.802123,0.087887,27
5,0.020346,0.011346,0.013333,0.003658,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.874254,0.874254,0.840114,0.928571,0.946153,0.892669,0.038961,1
6,0.029869,0.005177,0.008134,0.002581,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.852966,0.746799,0.644599,0.909115,0.872761,0.805248,0.096796,21
7,0.021957,0.001376,0.012519,0.002087,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.731250,0.765625,0.890114,0.803910,0.054851,23
8,0.014093,0.003820,0.014627,0.001996,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.857143,0.823129,0.874356,0.946153,0.878728,0.040797,3
9,0.025158,0.003651,0.009314,0.003868,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.852966,0.746799,0.644599,0.909115,0.872761,0.805248,0.096796,21
